In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import re
import pandas as pd
import numpy as np
import openpyxl

options = Options()
options.binary_location = r'C:\Program Files\Mozilla Firefox\firefox.exe'
driver = webdriver.Firefox(executable_path=r'C:\Users\dpriego\Anaconda3\geckodriver.exe', options=options)

wait_sec = 20
url_income_statement = 'https://finance.yahoo.com'
driver.get(url_income_statement)
driver.maximize_window()

#Obtener user agent
user_agent = driver.execute_script("return navigator.userAgent;")
print("User agent:", user_agent)

#Click boton aceptar coockies
element = driver.find_element_by_xpath("/html/body/div/div/div/div/form/div[2]/div[2]/button[1]")
driver.execute_script("arguments[0].click();", element)

#Click boton Industries
driver.implicitly_wait(wait_sec)
driver.find_element_by_xpath("//a[@title='Industries']").click()

#Click boton acciones de empresas tecnológicas
driver.implicitly_wait(wait_sec)
driver.find_element_by_xpath("//a[@title='Technology']").click()

#Obtenemos el listado de las distintas compañías
driver.implicitly_wait(wait_sec)
html_tech_sheet = BeautifulSoup(driver.page_source, 'html.parser')
element = driver.find_element_by_xpath("/html/body/div[1]/div/div/div[1]/div/div[2]/div/div/div[6]/div/div/section/div/div[2]/div[2]/button[3]")
rows = []
header = []
salir = False
table = html_tech_sheet.find('table')
while not salir:  
    for i, row in enumerate(table.find_all('tr')):
        if i == 0:
            header = [el.text.strip() for el in row.find_all('th')]
        else:
            rows.append([el.text.strip() for el in row.find_all('td')])
    element = driver.find_element_by_xpath("/html/body/div[1]/div/div/div[1]/div/div[2]/div/div/div[6]/div/div/section/div/div[2]/div[2]/button[3]")    
    if not element.is_enabled():
        salir = True
    driver.find_element_by_xpath("/html/body/div[1]/div/div/div[1]/div/div[2]/div/div/div[6]/div/div/section/div/div[2]/div[2]/button[3]").click()
    html_tech_sheet = BeautifulSoup(driver.page_source, 'html.parser')
    table = html_tech_sheet.find('table')

#Extraemos el identificador y nombre de cada empresa en un diccionaro para realizar el scraping de cada una de ellas:
companys = {}
for c in rows:
    if c[0] not in companys.keys():
        companys[c[0]] = c[1]
firstLoop = True
regexDate = re.compile('[@_!#?/\|}{~:]')
headers = ["Company", "Period", "Concept", "Value"]
df = pd.DataFrame(columns=headers)
values = []
i = 0
#for key, value in companys.items():
#Recorremos las empresas y obtenemos la información financiera:
for key in list(companys.keys())[0:3]:
    periodo = ["TTM"]
    ticker = key
    url_income_statement = 'https://finance.yahoo.com/quote/'+ticker+'/financials?p='+ticker
    driver.get(url_income_statement)
    
    #Click boton registrarse luego
    if firstLoop:      
        WebDriverWait(driver, wait_sec).until(EC.element_to_be_clickable((By.XPATH,
        "/html/body/div[1]/div/div/div[1]/div/div[4]/div/div/div[1]/div/div/div/div/div/section/button[2]"))).click()
        firstLoop = False

    #Click boton Expandir datos
    WebDriverWait(driver, 5)
    WebDriverWait(driver, wait_sec).until(EC.element_to_be_clickable((By.XPATH,
    "/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[2]/button/div/span"))).click()
    
    html_income_statement = BeautifulSoup(driver.page_source, 'html.parser')
    
    #Navegamos a balance-sheet
    WebDriverWait(driver, wait_sec).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 
    "a[href*='balance-sheet']"))).click()

    #Click boton Expandir datos
    WebDriverWait(driver, 5)
    WebDriverWait(driver, wait_sec).until(EC.element_to_be_clickable((By.XPATH,
    "/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[2]/button/div/span"))).click()
    
    html_balance_sheet = BeautifulSoup(driver.page_source, 'html.parser')

    #Navegamos a cash-flow
    WebDriverWait(driver, wait_sec).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 
    "a[href*='cash-flow']"))).click()
    
    #Click boton Expandir datos
    WebDriverWait(driver, 5)
    WebDriverWait(driver, wait_sec).until(EC.element_to_be_clickable((By.XPATH,
    "/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[2]/button/div/span"))).click()

    WebDriverWait(driver, 5)
    html_cash_flow = BeautifulSoup(driver.page_source, 'html.parser')

    #Obtenemos los datos de la tabla income_statement, los pasamos a un dataframe y finalmente los guardamos en un fichero
    income_statement = html_income_statement.findAll('span', attrs={"data-test":""})
    start_scraping = False
    values = []
    for index, val in enumerate(income_statement):
        if (val.text == "ttm"): start_scraping = True
        if (val.text == "People Also Watch"): start_scraping = False
        if (start_scraping == True): 
            if (regexDate.search(val.text) != None):
                    periodo.append(val.text)  
            elif (val.text.find(",")) > 0 or val.text=="0":
                values.append(val.text)            
                while len(values) != len(df.columns):
                       values.append(0) 
                df.loc[len(df)] = values
                df.loc[df.shape[0]-1,"Period"] = periodo[i]
                i+=1
                values.pop()
            else:    
                values = []
                i = 0
                values.insert(0,companys[key])  
                values.insert(1,"")
                values.append(val.text)
df.to_excel('FinanceData.xlsx', index=False)          
driver.quit()